# หัวข้อ : สร้าง RAG สำหรับภาษาไทย ด้วย OpenSource 

#### จัดทำโดยทีม [VulturePrime](https://vultureprime.com) 

## รายละเอียด 
การสร้าง RAG เริ่มต้นเป็น Use case ที่แพร่หลายสำหรับการนำ AI เข้ามาประยุกต์ใช้กับธุรกิจ
แต่เนื่องจากความขาดแคลนตัวอย่างที่เหมาะสมของแต่ละภาษา ทำให้เกิดปัญหา 2 ปัญหาได้แก่
1. English centric ตัวอย่างส่วนใหญ่จะถูกทำขึ้นมาโดยใช้ภาษาอังกฤษเป็นศูนย์กลาง ทำให้ภาษาอื่นนั้น จำเป็นต้องประยุกต์ Solution ขึ้นมาเองซึ่งทำให้ Developer ต้องลงทุนในการเรียนรู้เพิ่มขึ้นอย่างมหาศาล ไม่ว่าจะเป็นในเรื่องของ Tokenizer หรือ Text splitter 
2. OpenAI centric เนื่องจากการเชื่อมต่อกับ OpenAI API นั้นเป็นเรื่องที่ใช้ Effort ต่ำสุดและมีประสิทธิภาพสูงที่สุด ทำให้ AI และ Embedding Model อื่นนั้น กลายเป็น 3rd class citizen เมื่อนักพัฒนาอยากลดรายจ่ายโดยใช้ AI และ Embedding รายอื่น จึงเป็นเรื่องที่มีต้นทุนสูงอย่างมหาศาล (Switching cost)  

จากปัญหาทั้ง 2 ปัญหาทำให้เกิด AI Adoption ที่ช้ากว่าและแพงกว่าบริษัทที่ใช้ภาษาอังกฤษ​เป็นภาษาหลัก

จึงเป็นที่มาของการสร้าง Use case ด้วยโปรเจคตัวอย่าง เพื่อให้ Developer สามารถลด Learning curve ลงไปได้
และส่งเสริมให้มีความรวดเร็วในการนำ AI ไปใช้เพิ่มประสิทธิของ Feature ที่มีอยู่แล้วหรือสร้าง Feature ใหม่ขึ้นมา

## บทความเพิ่มเติม (ภาษาไทย)
- [Driving a Q&A Bot Project](https://www.vultureprime.com/blogs/driving-a-q-a-bot-project-a-product-owners-guide)

## Benefit 
- ราคาต่อ Character ที่ถูกกว่า 90 - 95% เมื่อเทียบกับ OpenAI
- เวลาในการประมวลผลที่น้อยลงเหลือเพียง 1/3 ถึง 1/4 (Float16 API หรือ SLA ตามต้องการ) 
- สามารถใช้งาน Offline หรือ Private เองได้

## ความท้าทาย
- การนับจำนวน Token ที่ไม่เท่ากันของภาษาอังกฤษและภาษาอื่น ทำให้ต้องสร้าง Custom Helper function เพื่อให้รองรับภาษาอื่น
- การตัดประโยคของภาษาไทยเมื่อเทียบกับภาษาอื่น
- การใช้งาน OpenSource เช่น Huggingface Embedding และ OpenAI API-like
- การใช้งาน Vector Database และทำความเข้าใจ Parameter ในการค้นหา

## Environment 
- [LlamaIndex](https://www.llamaindex.ai/) (Data Framework)
- [Weaviate](https://weaviate.io/) (Vector Database)
- [SeaLLM-7b](https://huggingface.co/SeaLLMs/SeaLLM-7B-Chat) (AI Model) ใช้งานผ่าน [Float16.cloud](https://float16.cloud)
- [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large) (Embedding Model)

## Flow 
1. บันทึกข้อมูลรูปแบบ String Text ไปยัง Vector Database
2. ค้นหาข้อมูลจาก Vector Database และนำไปใช้กับ AI Model

-------------------------------

#### 1.เก็บข้อมูลรูปแบบ String Text ไปยัง Vector Database

In [ ]:
#ทำการ Import Library ที่จำเป็น 

from llama_index import ServiceContext, StorageContext,VectorStoreIndex,Document
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.vector_stores import WeaviateVectorStore
from llama_index.node_parser import TokenTextSplitter
import weaviate 

#ทำการ Config API ที่ต้องการใช้
FLOAT16_API_KEY = 'float16-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
FLOAT16_CUSTOM_URL = 'https://api.float16.cloud/v1/llamaindex'
WEAVIATE_ENDPOINT = "https://XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

#กำหนดชื่อ Embedding model ที่รองรับภาษาไทย
embedding_model_name = "intfloat/multilingual-e5-large"

#กำหนดชื่อ Document สำหรับ String Text ที่ต้องการบันทึกใน Vector Database
document_name = 'FinOps_document_test1'

#กำหนดชื่อสำหรับ text key สำหรับการค้นหา
text_key = 'content'

#Download Embedding Model จาก Huggingface
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024)
#เชื่อมต่อกับ Vector Database
client = weaviate.Client(WEAVIATE_ENDPOINT)

#สร้าง VectorStore Node
vector_store = WeaviateVectorStore(weaviate_client = client,index_name = document_name,text_key = text_key)

In [ ]:
#String Text ที่ต้องการบันทึก อ้างอิงจากบทความ Cloud FinOps สำคัญต่อธุรกิจยุคดิจิทัลอย่างไร
#https://www.vultureprime.com/blogs/how-important-is-cloud-finops-for-businesses-in-the-digital-era 

Finops_principle = """II FinOps ?  
A FinOps (cloud Financial Operations)
FinOps Foundation เป็น Foundation ที่ได้รับการสนับสนุนจาก The Linux Foundation ในการช่วยสร้าง Foundation สำหรับการส่งเสริมและผลักดันการใช้งาน Cloud computing อย่างมีคุณค่า

B History
FinOps Foundation ก่อตั้งขึ้นมาตอน 2019 เกิดขึ้นหลังจากการมีอยู่ของ Cloud computing ประมาณ 10 ปี (AWS establish in 2006) ซึ่งถือว่าเป็น Foundation ที่มีอายุน้อยมาก (เทียบกับปี 2023) ฉะนั้นแล้วในส่วนของ Framework และ Best practice มีการอัพเดทอย่างรวดเร็วและเพิ่มขึ้นอย่างต่อเนื่อง เนื่องจาก use case ส่วนที่ยังไม่ได้ค้นพบยังมีอยู่มหาศาล
III Cloud in real life  
การใช้งาน Cloud นั้นเริ่มต้นนั้นจะดูเข้าใจง่าย แต่ความจริงแล้ววิธีคิดเงินของ Cloud นั้นมีความซับซ้อนและความเปลี่ยนแปลงสูง ซึ่งยังมีสิ่งที่ไม่เกี่ยวข้องกับวิธีคิดเงิน แต่เกี่ยวข้องกับความรับผิดชอบบางประการที่ถูกเปลี่ยนแปลงมาโดยไม่มีใครสังเกตุ
A Spending decision
ความรับผิดชอบด้านการจัดซื้อถูกย้ายมาให้กับทีม Engineer โดยตั้งใจหรือไม่ตั้งใจ ก็เป็นไปได้ทั้งสองกรณี โดยปกติแล้วการตัดสินใจในรายจ่ายเราจะมอบให้กับทีม Financial หรือทีม Procurement แต่เมื่อเป็น Cloud environment เรียบร้อยแล้ว คนที่เข้าใจ Cloud มากที่สุดกลับเป็นทีม Engineer ที่ไม่มีหน้าที่ในการรับผิดชอบการใช้จ่ายของบริษัท หรือเมื่อ Spending decision ยังอยู่กับทีม Financial คนที่ต้อง Commit รายจ่ายของ Cloud computing ก็ยังคงเป็นทีม Engineer

ถ้าไม่แก้ไขปัญหาจากเหตุการณ์ดังกล่าวจะเกิดเหตุการณ์ Engineer Commit รายจ่ายให้สูงเกินความจำเป็น หรือ การใช้งานบางอย่าง (Serverless) ส่งผลให้เกิดรายจ่ายที่ผิดปกติในเดือนนั้น ๆ โดยไม่ทราบสาเหตุ

วิธีแก้ไขของหลายบริษัทคือการให้ Engineer ลดรายจ่ายให้อยู่ในเพดานที่เหมาะสม

อ่านดูแล้วอาจเป็นวิธีการแก้ปัญหาที่ตรงจุด แต่วิธีดังกล่าวกลับเหมาะสมกับบริบทดั้งเดิม ไม่สมเหตุสมผลกับบริบทปัจจุบัน เพราะการกระจายอำนาจตัดสินใจให้กับทีมอื่นได้นั้นถือเป็นความสามารถที่ไม่เคยเกิดขึ้นมาก่อนและยังสอดคล้องกับการทำงานยุคใหม่ที่ใช้ Framework อย่างเช่น Agile ที่ต้องการให้เกิดความรวดเร็วในการตัดสินใจ

B Variable spending
การใช้งานบางบริษัทของผู้ให้บริการ Cloud computing ไม่ได้คิดค่าบริการตามจำนวนชั่วโมงที่ใช้งานอย่างที่เราเข้าใจ แต่การคิดค่าบริการอาจคิดจากจำนวนการเรียกใช้งานหรือจำนวนของ bandwidth เป็นต้น

ลักษณะการคิดค่าบริการดังกล่าวมีข้อดีคือเป็นการคิดค่าใช้จ่ายแบบ Variable cost เราสามารถนำไปใช้ในการคำนวณหา Cost Of Goods Sold (COGS) ได้ทันที

ส่วนข้อเสียคือถ้าทีม Financial ไม่เข้าใจที่มาของต้นทุนดังกล่าวแล้วไม่ได้ลงไว้ใน Variable cost หรือ COGS จะเกิดคำถามว่าทำไมค่าใช้จ่าย Cloud computing ในแต่ละเดือนนั้น ไม่เท่ากันและไม่สามารถพิจารณาค่าใช้จ่าย Cloud computing ล่วงหน้า

วิธีแก้ไขของหลายบริษัทคือไม่ใช้งาน Cloud computing ในส่วน Variable cost หรือไม่ต้องใช้งาน Cloud ไปเลย

C Scalable resource
การปรับจำนวนทรัพยากรได้ตลอดเวลานั้นสามารถส่งเสริมให้เกิดนวัตกรรมได้ตลอดเวลา เนื่องจากทีม Engineer สามารถเข้าถึงทรัพยากรได้อย่างรวดเร็วและเข้าถึงบางทรัพยากรที่ไม่เคยมีมาก่อน เช่น การเข้าถึง High performance computer, การเข้าถึง Computer cluster เป็นต้น สิ่งเหล่านี้จะทำให้ทีม Engineer สามารถทดลองนวัตกรรมใหม่ ๆ หรือสำรวจความเป็นไปได้ใหม่ ๆ โดยปราศจากข้อจำกัดด้านการจัดซื้อ

แต่ข้อเสียที่เกิดขึ้นเมื่อทีม Engineer สามารถเข้าถึงทรัพยากรเหล่านี้ได้โดยปราศจากการอนุมัติจากทีม Finance บางครั้งทีม Engineer อาจจะลืมหรือละเลยการใช้งานทรัพยากรเหล่านี้โดยเปิดทิ้งไว้ตลอดเวลา แต่ใช้งานจริงแค่เพียง 1% ของเวลาที่คิดค่าใช้จ่าย  ทำให้เกิดค่าใช้จ่ายส่วนเกินที่ไม่ควรเกิดขึ้น

ดังนั้นเพื่อเป็นการป้องกันปัญหาดังกล่าวจำเป็นต้องให้เกิดการตรวจสอบการใช้งานได้ตลอดเวลา เพื่อติดตามและป้องกันไม่ให้เกิดปัญหาดังกล่าว

VII  FinOps framework ถูกสร้างขึ้นมาโดยมีหัวข้อหลักทั้งหมด 5 หัวข้อ ดังนี้
Maturity
Principles
Personas
Phases
Domains
โดยแต่ละหัวข้อจะอธิบายถึงสถานะ, เครื่องมือ, กิจกรรม, ความเข้ากันได้ หรือ สิ่งขับเคลื่อน ซึ่งแต่ละหัวข้อสามารถแยกพิจารณาเมื่อไหร่ก็ได้ ไม่จำเป็นต้องเรียงลำดับและในแต่ละบริษัทไม่จำที่จะต้องให้น้ำหนักกับแต่ละหัวข้อเหมือนกันทุกบริษัท ขึ้นอยู่กับบริบทของบริษัทนั้น ๆ

เราขอเริ่มอธิบายจากหัวข้อแรก

A Maturity
Maturity เราอาจจะเรียกได้ว่าเป็นประสบการณ์ของทีม แต่ในที่นี้ขอเรียกว่าความเข้ากันได้ เนื่องจากบางบริษัททีมมีความเข้ากันได้อยู่แล้ว เคยทำงานร่วมกันด้วยความเข้าใจดีเยี่ยม ความเข้ากันได้ของทีมนั้น ๆ ก็จะสูงตาม

ในบริบทของ FinOps เราแบ่ง Maturity ออกเป็น 3 ช่วง ได้แก่

Crawl
ช่วงแรกเริ่มของการนำ FinOps เข้ามาประยุกต์เข้ากับบริษัทของตัวเอง ซึ่งเราสามารถประเมินได้ว่าบริษัทที่อยู่ในช่วงดังกล่าวได้ว่า กระบวนการทำงาน, กระบวนการรายงานผล ยังอยู่ในรูปแบบของ Manual หรือไม่ได้ใช้ระบบอัตโนมัติเข้ามาช่วย รวมถึงการทำงานบางส่วน เช่น Forecast ค่าใช้จ่าย ยังคงต้องคำนวนใหม่ทุกครั้ง ยังไม่ได้สร้างสูตรหรือโมเดลในการช่วยคำนวน ต้องให้ทีม Finance ประเมินทุก ๆ คร้ัง

Walk
บางบริษัทอาจจะเริ่มจากช่วงนี้เลยก็เป็นไปได้ สำหรับช่วง walk นั้นจะเรียกได้ว่าเป็นช่วงของ Semi-Automate process เนื่องจากทีมมีบุคคลที่มีความสามารถในการสร้าง Automate process ด้วยตัวเอง ไม่ว่าจะเป็นทีม Engineer, ทีม Finance หรือทีม Finance ทำให้งานบางประเภทนั้นมีความสามารถในการทำงานอย่างอัตโนมัติ Automate process ที่สร้างขึ้นมานั้นสามารถให้คนที่ไม่มีความเชี่ยวชาญที่เกี่ยวข้องสามารถใช้งานได้โดยไม่มีผู้เชี่ยวชาญ

Run
“ทีมที่เพียบพร้อมด้วยสหวิทยาการ” นี่คือคำนิยามแบบเข้าใจง่ายของช่วงนี้ เพราะทีมที่มีความเข้ากันได้สูง สามารถสร้าง Automate process ออกมาได้ตรงประเด็น จำเป็นต้องมีความเข้าใจทั้งในส่วนที่ตัวเองเชี่ยวชาญและความเข้าใจในส่วนที่นอกเหนือจากที่ตัวเองเชี่ยวชาญ เพื่อที่จะสร้างระบบเพื่อส่งมอบข้อมูลเพื่อนำไปใช้ในการตัดสินได้ตลอดเวลา (Realtime) และช่วยให้บุคคลอื่นสามารถตัดสินใจบนข้อมูลได้อย่างทันท่วงที ไม่จำเป็นต้องปรึกษากับทีมอื่น

B Principles
Principles คือสิ่งที่ระบุถึงสิ่งที่จำเป็นต้องบรรลุเมื่อต้องการนำ FinOps เข้ามาใช้ในบริษัท ส่วนของ Principles นั้นจะมีความใกล้เคียงกับ Framework ที่เกี่ยวข้องกับ Digital transformation อย่างเช่น Agile หรือ DevOps เนื่องจากต้องการให้บริษัทไม่จำเป็นต้องปรับตัวทั้งหมดเพื่อที่จะเริ่มนำ FinOps ไปใช้งาน โดย Principles นั้นมีอยู่ทั้งหมด 6 ข้อได้แก่

ทีมต้องทำงานร่วมกัน
ทุกคนต้องเป็นเจ้าของการใช้งาน Cloud computing ร่วมกัน
FinOps นั้นจะถูกขับเคลื่อนโดยทีมส่วนกลาง
Report ที่เกี่ยวข้อง (Engineer, Finance, Business) ต้องเข้าถึงได้และทันท่วงที
การตัดสินใจต้องถูกขับเคลื่อนโดยคุณค่าด้านธุรกิจผ่านการใช้งาน Cloud computing
ใช้ความได้เปรียบด้าน Variable cost จาก Cloud computing
C Personas
Personas ในที่นี้จะระบุถึงแรงจูงใจ, ปัญหา, ตัวชี้วัดและสิ่งที่ได้จาก FinOps ซึ่ง Personas นั้นจะถูกแบ่งออกเป็น 5 บุคคลที่เกี่ยวข้องได้แก่

1 FinOps Practitioner

บุคคลผู้ซึ่งต้องการนำ FinOps เข้ามาใช้กับกระบวนการทำงานปัจจุบันของบริษัท โดยเป็นผู้ที่มีความเข้าใจใน Personas ของผู้อื่นมากที่สุด นิยามอย่างง่ายคือ ทีม Finance มองว่า (Practitioner) เป็นคนจากทีม Engineer แต่ทีม Engineer มองว่าเป็นคนจากทีม Fiannce

2 Executive

ผู้บริหารคืออีกหนึ่ง Personas ที่ต้องเข้ามามีส่วนเกี่ยวข้องในเงื่อนของการคาดหวังผลลัพธ์และต้องมีส่วนช่วยในการสนับสนุน (Sponsor) การเปลี่ยนแปลงดังกล่าวให้เกิดขึ้นให้ได้  

3 Business

ทีม Business หรือ Product owner ผู้ซึ่งต้องดูแลการเติบโตและวางเป้าหมายให้ตัวผลิตภัฑณ์หรือบริการที่กำลังดูแลอยู่นั้น ส่งมอบมูลค่าแก่ลูกค้าได้อย่างครบถ้วนและตรงจุด เป็นผู้ซึ่งเข้าใจในส่วนของ User หรือ Customer ได้ดีที่สุดจากทั้ง 5 บุคคลที่เกี่ยวข้อง

4 Finance

ทีม Finance เป็นทีมที่ดูแลเกี่ยวกับการลงทุนในแต่ละโปรเจคและต้องคอยประเมินถึงค่าใช้จ่ายระยะยาว โดยต้องประเมินถึงความคุ้มทุนของโปรเจคที่ต้องลงทุนและประเมินว่าเมื่อถึงเวลาใดจะเป็นจุด Break event point

5 Engineer

บุคคลที่เป็นคนตัดสินความเป็นไปได้ของการสร้างผลิตภัฑณ์หรือบริการ ซึ่งความเป็นไปได้สำหรับผลิตภัฑณ์หรือบริการทางด้าน software นั้น ตั้งอยู่บนพื้นฐาน 3 สิ่งคือ Quality, Cost และ Time สามารถเลือกได้แค่ 2 ใน 3 เท่านั้น

D Phases
Phase กล่าวคือวงวนกระบวนการสำหรับ FinOps โดยกระบวนการดังกล่าวนั้นสามารถเริ่มจากกระบวนการใดก็ได้ ไม่จำเป็นต้องเริ่มจาก Inform แต่จำเป็นต้องทำตามลำดับเช่น เริ่มจาก Optimize จากนั้นเรียงไป Operate และ Inform ตามลำดับ ซึ่งเมื่อเราสามารถวนครบรอบได้แล้ว เราสามารถทำต่อไปได้ไม่มีที่สิ้นสุด เนื่องจากทุก ๆ ครั้งในการครบรอบสิ่งที่ทีมจะได้เพิ่มคือความเข้ากันได้ของทีม (Maturity)

Phase แบ่งออกด้วยกันทั้งหมด 3 phase ได้แก่

1 Inform

Phase inform จะเน้นผลลัพธ์ในการสร้างการเข้าถึงข้อมูล, การจัดการข้อมูล, การวัดผล, การกำหนดรายจ่าย และการ forecast รายจ่าย ในส่วนของ Inform จำเป็นต้องสร้างตัวชี้วัดโดยอาศัยข้อมูลจากทีมอื่นเข้ามาเป็นตัวแปรร่วมด้วย

2 Optimize

หลาย ๆ บริษัทมักอยากจะกระโดดเข้ามาใน Phase นี้เป็นส่วนแรก เนื่องจากเห็นผลได้รวดเร็วและมองเห็นถึงผลลัพธ์ได้ง่าย ในส่วนของ FinOps Foundation นั้นให้ความหมายของ Phase นี้เอาไว้ว่า เป็นช่วงที่ต้องมองหาความเป็นไปได้ในการลดต้นทุนและใช้ทรัพยากรที่มีอยู่ให้มีประสิทธิภาพสูงที่สุด หรือเข้าใจโดยง่ายว่า การ Optimize ทุกอย่างนั้นจะเกิดขึ้นแบบทฤษฎีก่อนนำไปให้ทีมที่เกี่ยวข้องนั้นนำไปใช้งานจริง

3 Operate

Phase นี้มักเป็นส่วนที่ถูกมองข้ามในการจำกัดความแต่ในการปฏิบัตินั้นเกิดขึ้นโดยไม่รู้ตัว ซึ่งการ Operate จะเป็นการเริ่มติดตามถึงตัวชี้วัดที่เราวางไว้, ติดตามถึงผลกระทบในส่วนของ Quality, Cost และ Time และในส่วนของการลงมือนำแผนการจาก Optimize นำไปใช้จริง

E Domain
Domain ระบุถึงกลุ่มกิจกรรมที่ให้ผลลัพธ์ออกมาในกลุ่มผลลัพธ์เดียวกัน โดย Domain นั้นจะถูกนำไปใช้ในแต่ Phase โดย Domain นั้นจะถูกกำหนดไว้ ณ ปัจจุบัน 6 Domain ได้แก่

1 Understanding cloud usage and cost

สำหรับ Domain นี้คือการรวมรวบข้อมูลที่เกี่ยวข้องกับการใช้งาน Cloud computing และนำข้อมูลที่ได้มานั้ดจัดให้อยู่ในรูปแบบที่สามารถเข้าถึงได้ ซึ่งข้อมูลดังกล่าวต้องตอบสนองต่อแต่ละ Personas อย่างครบถ้วน

2 Performance Tracking & Benchmarking

สำหรับการวัดผลและตั้งเป้าหมายจะถูกกำหนดขึ้น โดยเริ่มจากการตั้งงบประมาณ, การกำหนดผลลัพธ์ที่คาดหวัง หรือ ใช้ข้อมูลย้อนหลังเพื่อเป็นสมมติฐานการคำนวนรายจ่ายในอนาคต การวัดผลและการสร้างตัวชี้วัดทั้งหมดจะถูกสร้างใน Domain นี้

3 Real-Time Decision Making

การสร้างระบบสำหรับตัดสินใจแบบ Real time จะช่วยให้ผู้ที่มีส่วนได้ส่วนเสียทั้งหมดมีประสิทธิภาพในการรับมือกับสถานการณ์ต่าง ๆ ได้ดียิ่งขึ้น ไม่ว่าจะเป็นในด้านของเวลาในการตัดสินใจ หรือ คุณภาพในการตัดสินใจซึ่งต้องไปในทิศทางเดียวกันกับบริษัท

4 Cloud Rate Optimization

ภายใน Domain นี้จะกล่าวถึงการทำความเข้าใจเกี่ยวกับรูปแบบการคิดค่าบริการของผู้ให้บริการ Cloud computing และทำการปรับปรุงรายจ่าย Cloud computing ของบริษัทให้สอดคล้องกับรูปแบบการคิดค่าบริการของผู้ให้บริการ โดยคำนึงถึงข้อจำกัดที่แปรผันตามรูปแบบการคิดค่าบริการที่ต่างไป

5 Cloud Usage Optimization

Domain นี้ให้ความสำคัญถึงการใช้ทรัพยากรที่มีอยู่ให้คุ้มค่าที่สุด โดยการระบุและลงมือเปลี่ยนแปลงงานบางประเภทให้ประมวลผลให้ช่วงเวลาที่ Cloud computing นั้นไม่ได้ใช้งาน และรวมถึงการเปลี่ยนแปลงทรัพยากรที่มีอยู่ในเหมาะสมกับประเภทของงาน

6 Organizational Alignment
สำหรับ Domain นี้จะกล่าวถึงการที่บริษัทนั้นมีทิศทางในการใช้งาน Cloud computing ในทิศทางเดียวกันผ่าน Principles ของ FinOps ร่วมกัน เพื่อที่จะกำหนดวัฒนธรรมและนโยบายสำหรับบริษัทโดยไม่ขัดแย้งต่อวัฒนธรรมเดิมที่มีอยู่ของบริษัท
"""

In [ ]:
#กำหนดค่าของ Text Splitter ที่ต้องการใช้ 
#Chunk size กำหนดถึงความยาวของ Text ที่ต้องการตัด และ Chunk overlap หมายถึงการคาบเกี่ยวของ Text แต่ละชุดให้คาบเกี่ยวกับชุดก่อนหน้ามากน้อยแค่ไหน

text_parser = TokenTextSplitter.from_defaults(chunk_overlap=256,chunk_size=1024)

#ประมวลผล String Text ให้กลายเป็น List[String] โดย String มีขนาดตามที่กำหนดจากขั้นตอนก่อนหน้านี้
nodes = text_parser.get_nodes_from_documents(
    [Document(text=Finops_principle)], show_progress=True
)

In [ ]:
#กำหนดให้ใช้งาน Embedding ที่ทำการ Download มาจาก Huggingface
service_context = ServiceContext.from_defaults(embed_model=embed_model,llm=None)
#กำหนดให้ใช้งาน VectorDatabase ที่ทำการเชื่อมต่อกับ Weaviate
storage_context = StorageContext.from_defaults(vector_store = vector_store)

In [ ]:
#นำ List[String] ที่ได้จากขั้นตอนประมวลผล Text นำมาแปลงให้อยู่ในรูปของ List[Vector] 
#แปลง String เป็น Vector โดยใช้ Embedding Model intfloat/multilingual-e5-large
#หลังจากได้ List[Vector] และ List[String] ให้นำข้อมูลดังกล่าวเก็บไว้ยัง VectorDatabase ที่ได้เชื่อมต่อไว้แล้ว 
#โดยเก็บไว้ใน Document ชื่อ FinOps_document และมี Text_key เริ่มต้นด้วย "content"
VectorStoreIndex(nodes=nodes,storage_context=storage_context, service_context = service_context)

#### 2. ค้นหาข้อมูลจาก Vector Database และนำไปใช้กับ AI Model

In [ ]:
#ทำการ Import Library ที่จำเป็น 
from llama_index.llms import openai_like
from llama_index.readers.weaviate.reader import WeaviateReader
from llama_index import ServiceContext, ListIndex
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index.llms import OpenAI
from llama_index.indices.prompt_helper import PromptHelper
import weaviate

In [ ]:
#ทำการ Config API ที่ต้องการใช้
OPENAI_KEY = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
FLOAT16_API_KEY = 'float16-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
FLOAT16_CUSTOM_URL = 'https://api.float16.cloud/v1/llamaindex'
WEAVIATE_ENDPOINT = "https://XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"


In [ ]:
#ระบุ model ที่ต้องการใช้งาน 
#ณ​ วันที่ 5 มกราคม 2024 Float16 ยังคงรองรับแค่ seallm-7b เพียงตัวเลือกเดียว
llm_model_name = "seallm-7b-v2"

#กำหนดชื่อ Embedding model ที่รองรับภาษาไทย
embedding_model_name = "intfloat/multilingual-e5-large"

#กำหนดชื่อ Document สำหรับ String Text ที่ต้องการบันทึกใน Vector Database
document_name = 'FinOps_document_test1'

#กำหนดชื่อสำหรับ text key สำหรับการค้นหา
text_key = 'content'

#ระบุคำถามที่ต้องการถาม AI 
question = "FinOps คืออะไร"

#Download Embedding Model จาก Huggingface
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024)

#เชื่อมต่อกับ Float16
agent = openai_like.OpenAILike(
    api_key=FLOAT16_API_KEY,
    api_base=FLOAT16_CUSTOM_URL,
    model=llm_model_name,
    temperature=0.3,
    max_tokens=512,
    stream = True
)

#เชื่อมต่อกับ VectorDatabase
client = weaviate.Client(WEAVIATE_ENDPOINT)
reader = WeaviateReader(WEAVIATE_ENDPOINT)

In [ ]:
#สร้าง GraphQL query schema สำหรับการค้นหาข้อมูลที่เกี่ยวข้องจาก VectorDatabase
query_schema = """ 
{{
  Get{{
    {document_name}(
      nearVector: {{
        vector: {vector_question},  
        certainty: {certainty}
      }}
      limit : {limit}
    ) {{
      {text_key}
    }}
  }}
}}
"""

In [ ]:
#เปลี่ยนคำถาม (String) ให้กลายเป็น Vector
vector_question = embed_model._embed(question)[0]

In [ ]:
#ทำการ map ตัวแปรเข้ากับ Schema
query = query_schema.format(
  document_name=document_name,
  vector_question=vector_question,
  certainty=0.9,
  text_key=text_key,
  limit = 3
)

#ส่งคำสั่ง query ไปยัง VectorDatabase
documents = reader.load_data(graphql_query=query, separate_documents=True)

#### กำหนด Service ที่เกี่ยวข้องกับ AI 

llm คือการกำหนดให้ใช้งาน LLM ตัวใดหรือผู้ให้บริการเจ้าใดสำหรับการรับ Prompt เพื่อให้ตอบคำถาม 

embed_model คือการกำหนดให้ใช้งาน Embedding ตัวใด สำหรับการแปลง Text ให้เป็น Vector 

prompt_helper คือตัวช่วยจิปาถะที่เกี่ยวข้องกับการใช้งาน LLM เช่น การทำให้ Prompt ที่ส่งไปยังผู้ให้บริการมีขนาดไม่เกินที่กำหนด (max_context) 

หรือ เมื่อ Prompt มีขนาดยาวเกินไปก็จะทำการ "ตัดข้อมูล" โดยอัตโนมัติ 

สำหรับ context_window ที่เรากำหนดขึ้นมานั้นมีขนาด 10,000 token ซึ่งการนับ Token โดย Default นั้น ใช้งาน Tiktoken หรือ OpenAI ในการนับ Token 

ทำให้ไม่สอดคล้องกับ SeaLLM-7b เนื่องจาก SeaLLM-7b นับ Token ต่างกับ OpenAI 

ส่งผลให้เมื่อใช้งาน context_window default จะทำให้ข้อมูลขาดหายไปบางช่วง 

DEFAULT context_window = 3900 #token

In [ ]:
service_context = ServiceContext.from_defaults(llm=agent,embed_model=embed_model,prompt_helper=PromptHelper(context_window=10000))

In [ ]:
#สร้าง index สำหรับการค้นหาข้อมูลจาก ผลลัพธ์จาก query
index = ListIndex.from_documents(documents, service_context=service_context)

#### กำหนด Prompt pipeline สำหรับ RAG 

Default สำหรับ Llamaindex จะเป็นการ "refine" ซึ่ง refine จะเป็นขั้นตอนการทำงาน 2 ขั้นตอนต่อกัน
1. Question and Answer เพื่อให้ LLM ที่กำหนดไว้ใน Service_context ตอบคำถาม
2. Refine เพื่อให้คำตอบที่ได้ออกมานั้นสวยงามมากยิ่งขึ้น

ทำให้เสียค่าใช้จ่ายในการ Query 2 ครั้ง

ดังนั้นจึงเปลี่ยนเป็น "simple_summarize" จะลดขั้นตอนลงเหลือแค่ขั้นตอนเดียวนั้นคือ Question and Answer

In [ ]:
#กำหนด Prompt pipeline สำหรับ RAG 
response_synthesizer = get_response_synthesizer(
      streaming=True,response_mode="simple_summarize", service_context=service_context
)

In [ ]:
#สร้าง Query engine จาก index ที่เราได้สร้างจาก ผลลัพธ์จาก Query และเปลี่ยน Response ให้ทำงานแค่ 1 ขั้นตอน
query_engine = index.as_query_engine(
    streaming=True,
    response_synthesizer=response_synthesizer,
    service_context=service_context,
)

#ค้นหาผลลัพธ์จาก RAG
result = query_engine.query(question)
result.print_response_stream()

In [ ]:
# #เปรียบเทียบผลลัพธ์กับ gpt-3.5-turbo เพื่อดูความแตกต่าง
# openai_LLM = OpenAI(model="gpt-3.5-turbo", temperature=0.3, max_tokens=512,api_key=OPENAI_KEY)
# openAI_service_context = ServiceContext.from_defaults(llm=openai_LLM,embed_model=embed_model)
# response_synthesizer = get_response_synthesizer(
#   response_mode="simple_summarize", service_context=openAI_service_context
# )
# openai_query_engine = index.as_query_engine(
#   response_synthesizer=response_synthesizer,
#   service_context=openAI_service_context,
# )
# result = openai_query_engine.query(question)